In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

%cd /content/drive/MyDrive/YOLO_Testing

Mounted at /content/drive
/content/drive/MyDrive/YOLO_Testing


In [2]:
import torch
from IPython.display import Image, clear_output  # to display images

print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Setup complete. Using torch 2.1.0+cu118 (Tesla T4)


In [3]:
%cd /content/drive/MyDrive/YOLO_Testing/yolov5

/content/drive/MyDrive/YOLO_Testing/yolov5


In [ ]:
def read_label_file(file):
  label = []
  f = open(file, "r")
  lines = f.readlines()
  for line in lines:
    elements = list(map(float, line.split(' ')))
    label.append(elements)
  return label


def get_list_labels(results, sizes):
  import pandas as pd

  pandas_results = []
  for i in range(len(results)):
    pandas_result = results[i].pandas().xyxy[0]
    # Normalize xyxy
    pandas_result.iloc[:, 0] = pandas_result.iloc[:, 0] / sizes[0]
    pandas_result.iloc[:, 2] = pandas_result.iloc[:, 2] / sizes[0]
    pandas_result.iloc[:, 1] = pandas_result.iloc[:, 1] / sizes[1]
    pandas_result.iloc[:, 3] = pandas_result.iloc[:, 3] / sizes[1]
    pandas_results.append(pandas_result)

  all_results = pd.concat(pandas_results, axis=0)
  new_cols = ['class', 'xmin', 'ymin', 'xmax', 'ymax', 'confidence']
  all_results = all_results[new_cols]
  list_results = all_results.values.tolist()

  return list_results


def get_labels_after_agnostic_nms(labels, iou_threshold=0.45):
  from torchvision.ops import nms
  torch_label = torch.tensor(labels)
  pred_boxes, pred_scores = torch_label[:, 1:5], torch_label[:, 5]
  boxes_index = nms(boxes=pred_boxes, scores=pred_scores, iou_threshold=iou_threshold)

  list_of_indices = boxes_index.tolist()

  # Change format from xyxy_norm to xywh_norm
  for i, label in enumerate(labels):
    label[1] = (label[1] + label[3]) / 2
    label[2] = (label[2] + label[4]) / 2
    label[3] = (label[3] - label[1]) * 2
    label[4] = (label[4] - label[2]) * 2

  return [labels[i] for i in list_of_indices]


def write_list_to_file(lst, file_name, output_folder):
  with open(output_folder + file_name, 'w') as fp:
    for label in lst:
      for element in label:
        fp.write(str(element))
        fp.write(' ')
      fp.write('\n')
  fp.close()
  return output_folder + file_name


def save_image_with_prediction(label_path, img_path, save_dir):
  from utils import general_capstone
  import cv2
  from pathlib import Path

  with open(label_path, 'r') as f:
    bboxes = f.read().split('\n')
    print('Labels loaded from ', label_path)

  image = cv2.imread(img_path)

  img_to_save = general_capstone.make_image_wbbox(image, bboxes)

  save_path = save_dir+'/'+ Path(img_path).stem + '.jpg'
  cv2.imwrite(save_path, img_to_save)
  print(f'Image saved to {save_path}')


def recognize_IoT_objects(model, image_path):

  from PIL import Image
  # Image sizes configuration
  SIZE_1 = 640
  SIZE_2 = 1536
  IOU_THRESHOLD = 0.45
  OUTPUT_FOLDER = '/content/drive/MyDrive/YOLO_Testing/yolov5/runs/detect/our_exp/'
  LABEL_FILE_NAME = 'nms_label.txt'

  # Get predictions
  image = Image.open(image_path)
  img_size = image.size
  result_1 = model(image, size=SIZE_1)
  result_2 = model(image, size=SIZE_2)
  timestamp_2 = datetime.now()

  # Handle predictions into one single list
  list_results = get_list_labels([result_1, result_2], img_size)

  # Get agnostic nms result and write the label file
  results_after_nms = get_labels_after_agnostic_nms(list_results, IOU_THRESHOLD)
  nms_label_path = write_list_to_file(results_after_nms, LABEL_FILE_NAME, OUTPUT_FOLDER)

  # Save image with prediction
  save_image_with_prediction(nms_label_path, image_path, OUTPUT_FOLDER)
  return timestamp_2, datetime.now()

In [ ]:
from datetime import datetime
image_path = '/content/drive/MyDrive/YOLO_Testing/test_nov15_existediamges/1698811617178.jpg'
timestamp_0 = datetime.now()
model = torch.hub.load('.', 'custom', path='/content/drive/MyDrive/YOLO_Testing/yolov5/runs/train/exp/weights/best.pt', source='local')
timestamp_1 = datetime.now()

YOLOv5 🚀 v7.0-211-g94e943e Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 416 layers, 140172724 parameters, 0 gradients, 208.3 GFLOPs
Adding AutoShape... 


In [ ]:
timestamp_2, timestamp_3 = recognize_IoT_objects(model, image_path)
timelen_load = (timestamp_1 - timestamp_0).total_seconds() * 1000
timelen_infer = (timestamp_2 - timestamp_1).total_seconds() * 1000
timelen_merge = (timestamp_3 - timestamp_2).total_seconds() * 1000
timelen_total = (timestamp_3 - timestamp_0).total_seconds() * 1000
print(f'Speed: {timelen_load:.1f}ms load-model, {timelen_infer:.1f}ms infer, {timelen_merge:.1f}ms merge, TOTAL: {timelen_total:.1f}ms')

Labels loaded from  /content/drive/MyDrive/YOLO_Testing/yolov5/runs/detect/our_exp/nms_label.txt
Shape of the output image is [w, h]:  4000 2250
Image saved to /content/drive/MyDrive/YOLO_Testing/yolov5/runs/detect/our_exp//1698811617178.jpg
Speed: 3067.8ms load-model, 483.8ms infer, 747.2ms merge, TOTAL: 4298.8ms


In [ ]:
!python train.py --img 1280 --batch 16 --epochs 300 --data /content/drive/MyDrive/YOLO_Testing/Our_IoT_Aug/data.yaml\
                          --weights yolov5m6.pt --freeze 10 --patience 20\
                          --hyp hyp.scratch-high.yaml

Traceback (most recent call last):
  File "/content/drive/MyDrive/YOLO_Testing/yolov5/train.py", line 61, in <module>
    from utils.loggers import Loggers
  File "/content/drive/MyDrive/YOLO_Testing/yolov5/utils/loggers/__init__.py", line 14, in <module>
    from utils.loggers.clearml.clearml_utils import ClearmlLogger
  File "<frozen importlib._bootstrap>", line 1027, in _find_and_load
  File "<frozen importlib._bootstrap>", line 1006, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 688, in _load_unlocked
  File "<frozen importlib._bootstrap_external>", line 879, in exec_module
  File "<frozen importlib._bootstrap_external>", line 975, in get_code
  File "<frozen importlib._bootstrap_external>", line 1074, in get_data
KeyboardInterrupt
